In [1]:
#default_exp trainfull3d_labels

In [2]:
#export
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

Loading imports


In [3]:
torch.cuda.set_device(2)

## 3d dataset with tfms

In [4]:
dls = get_3d_dls_aug(Meta.df_comb, sz=128, bs=32, grps=Meta.grps_stg1)

## 3d model

In [5]:
#export
class Flat3d(Module):
    def forward(self, x): return x.view(x.shape[0],-1)

In [6]:

#export
class XResNet3D(nn.Sequential):
    @delegates(ResBlock)
    def __init__(self, block, expansion, layers, p=0.0, c_in=3, c_out=1000, stem_szs=(32,32,64),
                 widen=1.0, sa=False, act_cls=defaults.activation, **kwargs):
        store_attr(self, 'block,expansion,act_cls')
        stem_szs = [c_in, *stem_szs]
        stem = [ConvLayer(stem_szs[i], stem_szs[i+1], stride=2 if i==0 else 1, act_cls=act_cls, ndim=3)
                for i in range(3)]

        block_szs = [int(o*widen) for o in [64,128,256,512] +[256]*(len(layers)-4)]
        block_szs = [64//expansion] + block_szs
        blocks = [self._make_layer(ni=block_szs[i], nf=block_szs[i+1], blocks=l,
                                   stride=1 if i==0 else 2, sa=sa and i==len(layers)-4, **kwargs)
                  for i,l in enumerate(layers)]
        super().__init__(
            Batchify(),
            *stem, nn.MaxPool3d(kernel_size=3, stride=2, padding=1),
            *blocks,
            ConvLayer(512,512,stride=2,ndim=3), # 8
            ConvLayer(512,1024,stride=2,ndim=3), # 4
            ConvLayer(1024,1024,stride=2,ndim=3), # 2
            nn.AdaptiveAvgPool3d((1, 1, 1)), Batchify(), Flat3d(), nn.Dropout(p),
            nn.Linear(1024*expansion, c_out),
#             nn.Linear(block_szs[-1]*expansion, c_out),
        )
        init_cnn(self)

    def _make_layer(self, ni, nf, blocks, stride, sa, **kwargs):
        return nn.Sequential(
            *[self.block(self.expansion, ni if i==0 else nf, nf, stride=stride if i==0 else 1,
                      sa=sa and i==(blocks-1), act_cls=self.act_cls, **kwargs)
              for i in range(blocks)])

In [7]:
#export
def xres3d(**kwargs):
    m = XResNet3D(ResBlock, expansion=1, layers=[2, 2,  2, 2], c_out=6, ndim=3, **kwargs)
    init_cnn(m)
    return m

In [8]:
xb,yb = dls.one_batch()
xb.shape, yb.shape

(torch.Size([32, 60, 3, 128, 128]), torch.Size([32, 60, 6]))

In [9]:
m = xres3d().cuda()

In [10]:
learn = get_learner(dls, m, get_loss())
# learn.to_fp16()

In [11]:
xb, yb = dls.one_batch()
with torch.no_grad():
    print(learn.model(xb).shape)

torch.Size([32, 6])


In [12]:
#export
class RowLoss(Callback):
    def __init__(self): super().__init__()

    def after_pred(self):
        learn = self.learn
        learn.pred = learn.pred
        learn.yb = (learn.yb[0].max(dim=1)[0],)

In [13]:
learn.add_cb(RowLoss())

In [14]:
learn.summary()

XResNet3D (Input shape: ['32 x 60 x 3 x 128 x 128'])
Layer (type)         Output Shape         Param #    Trainable 
Batchify             32 x 3 x 60 x 128 x 128 0          False     
________________________________________________________________
Conv3d               32 x 32 x 30 x 64 x 64 2,592      True      
________________________________________________________________
BatchNorm3d          32 x 32 x 30 x 64 x 64 64         True      
________________________________________________________________
ReLU                 32 x 32 x 30 x 64 x 64 0          False     
________________________________________________________________
Conv3d               32 x 32 x 30 x 64 x 64 27,648     True      
________________________________________________________________
BatchNorm3d          32 x 32 x 30 x 64 x 64 64         True      
________________________________________________________________
ReLU                 32 x 32 x 30 x 64 x 64 0          False     
______________________________

In [ ]:
do_fit(learn, 1, 1e-2)

## Export

In [34]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_metadata.ipynb.
Converted 01_preprocess.ipynb.
Converted 01_preprocess_mean_std.ipynb.
Converted 02_train.ipynb.
Converted 03_train3d.ipynb.
Converted 03_train3d_01_train3d.ipynb.
Converted 03_train3d_01b_train_lstm.ipynb.
Converted 03_train3d_02_train_3d_head.ipynb.
Converted 03_train3d_02_train_lstm_head.ipynb.
Converted 03_train3d_02c_train_lstm_head_2ndPlace.ipynb.
Converted 03_trainfull3d.ipynb.
Converted 03_trainfull3d_labels.ipynb.
This cell doesn't have an export destination and was ignored:
e
Converted 03_trainfull3d_labels_01_partial3d.ipynb.
Converted 04_trainSeq_01_lstm.ipynb.
Converted 04_trainSeq_02_transformer.ipynb.
Converted 04_trainSeq_03_lstm_seutao.ipynb.
Converted 05_train_adjacent.ipynb.
Converted 05_train_adjacent_01_5c_adj_win.ipynb.
Converted 05_train_adjacent_01_5slice.ipynb.
Converted 05_train_adjacent_02_3c.ipynb.
Converted 05_train_adjacent_02_3c_stg1.ipynb.
Converted 06_seutao_features.ipynb.
Converted 06_seutao_features_01_simple_lstm_20ep.ip